In [1]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix, plot_stat_map
import os
import pandas as pd
import pickle
import re

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/01_level1')
from level1_utils import run_level1, make_level1_design_matrix, get_events

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/'
behavior_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/behavioral_data/all_trials.csv'
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/")

subnum = '01'
runnum='4'

In [3]:
fn = os.path.join(data_path, 'sub-%s/func/sub-%s_task-bundles_run-%s_events.tsv' %(subnum, subnum, runnum))
events = pd.read_csv(fn, sep='\t')

# Read in behavioral data with modeled value and RPE estimates
behavior = pd.read_csv(behavior_path)

# Extract the correct subnum and runnum from behavioral data
run_behavior = behavior.query('subnum == %d & session == %d'%(int(subnum), int(runnum)))

In [13]:
cond_valChosen = events.query('trial_type == "stimulus"')[['onset', 'duration']].reset_index(drop=True)
cond_valChosen['trial_type'] = 'valChosen'
cond_valChosen['modulation'] = run_behavior['leftbundleValAdv'].sub(run_behavior['leftbundleValAdv'].mean()).reset_index(drop=True)

In [43]:
np.where(run_behavior['choiceLeft'], run_behavior['leftBundleVal'], 0)

array([ 0.        ,  0.65292193,  0.        ,  0.        ,  0.        ,
        0.49927559,  0.        ,  0.9       ,  0.86323291,  0.39362291,
        0.        ,  0.        ,  0.78866981,  0.17660755,  0.        ,
        0.        ,  0.2526012 , -0.00390182,  0.        ,  0.65292193,
        0.78855327,  0.        ,  0.09124038,  0.        ,  0.        ,
        0.6989118 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.01130719,  0.        ,  0.        ,  0.        ,
        0.7       ,  0.        ,  0.7       ,  0.47699693,  0.        ,
        0.89854838,  0.        ,  0.61344788,  0.        ,  0.        ,
        0.00628063,  0.02506939,  0.        ,  0.1       ,  0.24839909,
        0.        ,  0.44711837,  0.        ,  0.        ,  0.67241005,
        0.        ,  0.        ,  0.        ,  0.12620794,  0.        ])

In [44]:
np.where(run_behavior['choiceLeft']==0, run_behavior['rightBundleVal'], 0)

array([ 0.36301191,  0.        ,  0.19441845,  0.44951237, -0.00297256,
        0.        ,  0.10145669,  0.        ,  0.        ,  0.        ,
        0.5       ,  0.49486593,  0.        ,  0.        ,  0.45064671,
        0.49342466,  0.        ,  0.        ,  0.49927561,  0.        ,
        0.        ,  0.38805426,  0.        ,  0.4816744 ,  0.29426278,
        0.        ,  0.1262594 ,  0.11920142,  0.10749792, -0.01040963,
        0.10261048,  0.        ,  0.13581906,  0.3598902 ,  0.49357104,
        0.        ,  0.48115561,  0.        ,  0.        ,  0.29945283,
        0.        ,  0.36303784,  0.        ,  0.49355755,  0.10155225,
        0.        ,  0.        ,  0.29425784,  0.        ,  0.        ,
        0.04265448,  0.        ,  0.1262072 ,  0.47694865,  0.        ,
        0.49909096,  0.43808441,  0.49355741,  0.        ,  0.04178214])

In [ ]:

design_matrix= make_level1_design_matrix(subnum, runnum, data_path, behavior_path, hrf_model = 'spm', drift_model='cosine')
plot_design_matrix(design_matrix)

In [ ]:
# run_level1(subnum, data_path, behavior_path, out_path)

In [ ]:
# contrast_id = 'valDiff'
# zmap = nib.load(os.path.join(out_path, 'sub-%s/contrasts/sub-%s_run-%s_%s.nii.gz'%(subnum, subnum, runnum, contrast_id)))
# plot_stat_map(zmap, threshold=3.0, title='sub-%s_run-%s_%s' % (subnum, runnum, contrast_id))